In [2]:
import pandas as pd

# Load the dataset
df = pd.read_csv("../data/raw/teachers_db_practice.csv")

# Take a quick look
print(df.shape)
df.head()


(1228, 16)


,Unnamed: 0,area,area_code,position,full_info,id,description,tags,published,size,width,height,orientation,site,gender,alias
0,0,Architecture & Design,40358,NaN,<p> has worked as a designer for the last dec...,184670.0,NaN,[],2025-06-16T11:48:18.952Z,22834.0,500.0,500.0,S,global,0,Appius Aemilius Agricola
1,1,Economics,17166,NaN,<p>Mr. Madgar has been teaching economics par...,16106.0,Close-up portrait of a smiling man with a bear...,"['man', 'portrait', 'smiling', 'beard', 'close...",2022-11-18T07:21:20.407Z,7180.0,170.0,170.0,S,global,0,Appius Aemilius Cicero
2,2,Private & Business Law,40353,NaN,<p>Lawyer with broad experience in Market Regu...,25635.0,NaN,[],2023-03-17T12:44:46.932Z,14699.0,128.0,128.0,S,global,0,Appius Aemilius Crassus
3,3,Economics,17166,Adjunct professor,<p> is a seasoned leader with a proven track r...,37534.0,A professional portrait of a smiling woman wit...,"['woman', 'portrait', 'smiling', 'blond hair',...",2024-02-15T12:47:08.017Z,79371.0,500.0,500.0,S,global,1,Flavia Prisca
4,4,Science & Technology,40359,NaN,"<p> Carrio is a seasoned technology leader, re...",182500.0,NaN,[],2025-04-07T10:15:21.908Z,15906.0,500.0,500.0,S,global,0,Appius Aemilius Scipio


In [3]:
df.columns
print(df.loc[0, "full_info"])

<p>  has worked as a designer for the last decade in roles spanning a variety of disciplines from graphics to product to interiors. In 2017, , became studio director at Vidivixi, a furniture and interiors design practice based in Mexico City. After leaving in 2023 and relocating to Spain he opened a new design studio with a focus on bespoke, contemporary design-led furniture. </p><h4>Corporate Experience</h4><p>• Studio Director, A&amp;M Studio, Spain, 2023 – Present</p><p>• Studio Director, Vidivixi, Mexico, 2017 – 2023</p><p>• Associate, Becquerel Capital, Mexico, 2014 – 2017</p><p>• Design Partner, The Hub, Hong Kong, 2013 – 2014</p><h4>Academic Background</h4><p>• Bachelor in Graphic Design, Camberwell College of Arts UAL, U.K., 2013</p>


In [5]:
import re

# Remove HTML tags and extra whitespace
df["clean_text"] = (
    df["full_info"]
    .fillna("")
    .apply(lambda x: re.sub(r"<.*?>", " ", x))
    .apply(lambda x: re.sub(r"\s+", " ", x).strip())
)

df["clean_text"].head(3).values

array(['has worked as a designer for the last decade in roles spanning a variety of disciplines from graphics to product to interiors. In 2017, , became studio director at Vidivixi, a furniture and interiors design practice based in Mexico City. After leaving in 2023 and relocating to Spain he opened a new design studio with a focus on bespoke, contemporary design-led furniture. Corporate Experience • Studio Director, A&amp;M Studio, Spain, 2023 – Present • Studio Director, Vidivixi, Mexico, 2017 – 2023 • Associate, Becquerel Capital, Mexico, 2014 – 2017 • Design Partner, The Hub, Hong Kong, 2013 – 2014 Academic Background • Bachelor in Graphic Design, Camberwell College of Arts UAL, U.K., 2013',
       'Mr. Madgar has been teaching economics part time while working in various professional roles across a wide array of industries. He taught undergraduate microeconomics and macroeconomics at Kent State University for 5 years before advancing to teach economic environment &amp; country ec

In [ ]:
from transformers import pipeline

# Load a pre-trained NER model
ner = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")

# Test on small sample of dataset
sample_text = df.loc[1, "clean_text"]
ner_results = ner(sample_text[:1000])  # limit to 1000 chars just to test
ner_results[:10]



config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use mps:0


[{'entity_group': 'PER',
  'score': np.float32(0.7767943),
  'word': 'Madgar',
  'start': 4,
  'end': 10},
 {'entity_group': 'ORG',
  'score': np.float32(0.99842983),
  'word': 'Kent State University',
  'start': 189,
  'end': 210},
 {'entity_group': 'ORG',
  'score': np.float32(0.98850226),
  'word': 'IE',
  'start': 327,
  'end': 329},
 {'entity_group': 'LOC',
  'score': np.float32(0.9988895),
  'word': 'US',
  'start': 506,
  'end': 508},
 {'entity_group': 'MISC',
  'score': np.float32(0.9949227),
  'word': 'Fortune 100',
  'start': 632,
  'end': 643},
 {'entity_group': 'ORG',
  'score': np.float32(0.9992545),
  'word': 'GE',
  'start': 662,
  'end': 664},
 {'entity_group': 'ORG',
  'score': np.float32(0.99678975),
  'word': 'Halliburton',
  'start': 669,
  'end': 680},
 {'entity_group': 'ORG',
  'score': np.float32(0.9991283),
  'word': 'Millwood Inc',
  'start': 704,
  'end': 716}]

In [7]:
from tqdm import tqdm
tqdm.pandas()

df["entities"] = df["clean_text"].progress_apply(lambda x: ner(x[:2000]))
df["entities"].head(3)


100%|██████████| 1228/1228 [01:21<00:00, 14.99it/s]


0    [{'entity_group': 'ORG', 'score': 0.97535497, ...
1    [{'entity_group': 'PER', 'score': 0.6852415, '...
2    [{'entity_group': 'MISC', 'score': 0.9922315, ...
Name: entities, dtype: object

In [8]:
from rapidfuzz import process, fuzz

def normalize_entities(entities):
    cleaned = []
    for ent in entities:
        word = ent["word"].strip()
        # Remove stray punctuation and unify case
        word = word.replace(".", "").replace(",", "").title()
        cleaned.append((ent["entity_group"], word))
    return cleaned

df["normalized_entities"] = df["entities"].apply(normalize_entities)
df["normalized_entities"].head(2).values



array([list([('ORG', 'Vidivixi'), ('LOC', 'Mexico City'), ('LOC', 'Spain'), ('ORG', 'A & Am'), ('ORG', 'M Studio'), ('LOC', 'Spain'), ('LOC', 'Vidiv'), ('ORG', '##Ixi'), ('LOC', 'Mexico'), ('ORG', 'Becquerel Capital'), ('LOC', 'Mexico'), ('ORG', 'The'), ('LOC', 'Hub'), ('LOC', 'Hong Kong'), ('ORG', 'Design'), ('ORG', 'Camberwell College Of Arts Ual'), ('LOC', 'U K')]),
       list([('PER', 'Madgar'), ('ORG', 'Kent State University'), ('ORG', 'Ie'), ('LOC', 'Us'), ('MISC', 'Fortune 100'), ('ORG', 'Ge'), ('ORG', 'Halliburton'), ('ORG', 'Millwood Inc'), ('ORG', 'Millwood Inc'), ('LOC', 'Usa'), ('ORG', 'Per'), ('LOC', '##Masteelisa'), ('LOC', 'Usa'), ('ORG', 'Jet Research Center'), ('LOC', 'Halliburton'), ('LOC', 'Usa'), ('ORG', 'Sigma Black Belt'), ('ORG', 'Cameron ‘ A Schlumberger Co ’'), ('LOC', 'Usa'), ('ORG', 'Process Technology'), ('ORG', 'Millwood Inc'), ('LOC', 'Usa'), ('ORG', 'Sigma Black Belt'), ('ORG', 'Mactac'), ('ORG', 'Bemis Co ’'), ('LOC', 'Usa'), ('ORG', 'Six Sigma / Lea'),

In [15]:
import re

LOCATION_MAP = {
    "us": "United States",
    "usa": "United States",
    "u.s.": "United States",
    "u.k.": "United Kingdom",
    "uk": "United Kingdom",
    "spain": "Spain",
    "madrid": "Madrid",
    "mexico": "Mexico",
    "mexico city": "Mexico City",
    "london": "London",
    "paris": "Paris",
    "france": "France",
    "germany": "Germany",
    "italy": "Italy",
    "portugal": "Portugal",
    "barcelona": "Barcelona",
    "new york": "New York",
}

ORG_FIXES = {
    # IE ecosystem
    "ie": "IE",
    "ie university": "IE",
    "ie business school": "IE",
    "ie law school": "IE",
    "instituto de empresa": "IE",
    "ie school of global and public affairs": "IE",
    # Spanish universities
    "universidad autonoma de madrid": "Universidad Autónoma de Madrid",
    "uam": "Universidad Autónoma de Madrid",
    "universidad complutense de madrid": "Universidad Complutense de Madrid",
    "universidad carlos iii de madrid": "Universidad Carlos III de Madrid",
    "universidad politecnica de madrid": "Universidad Politécnica de Madrid",
    "universidad de navarra": "Universidad de Navarra",
    "universidad pontificia comillas": "Universidad Pontificia Comillas",
    "comillas university": "Universidad Pontificia Comillas",
    "icade": "Universidad Pontificia Comillas",
    "iese business school": "IESE Business School",
    # companies / misc
    "a & am": "A&M Studio",
    "a&m": "A&M Studio",
    "am studio": "A&M Studio",
}

GENERIC_ORG_WORDS = {
    "academic", "academic exp", "academ", "experience", "exp",
    "university", "universidad", "engineering", "design",
    "academy", "school", "faculty", "department", "college",
    "education", "institute", "business", "finance", "management",
    "administration", "economics", "marketing", "law", "science",
    "technology", "research", "professor", "lecturer"
}

def normalize_entities(entities):
    cleaned = []
    for ent in entities:
        text = ent["word"].lower().strip()
        text = re.sub(r"[^a-z0-9&.\sáéíóúüñ]", "", text)
        text = re.sub(r"\s+", " ", text).strip()

        if len(text) < 3:
            continue

        if ent["entity_group"] == "LOC":
            text = LOCATION_MAP.get(text, text.title())

        elif ent["entity_group"] == "ORG":
            text = ORG_FIXES.get(text, text.title())

            # drop headings and generic garbage
            if (
                text.lower() in GENERIC_ORG_WORDS
                or len(text.split()) == 1 and text.lower() not in [v.lower() for v in ORG_FIXES.values()]
            ):
                continue

        else:
            text = text.title()

        cleaned.append((ent["entity_group"], text))
    return cleaned

df["normalized_entities"] = df["entities"].apply(normalize_entities)




from collections import Counter

org_counts = Counter([e[1] for row in df["normalized_entities"] for e in row if e[0] == "ORG"])
loc_counts = Counter([e[1] for row in df["normalized_entities"] for e in row if e[0] == "LOC"])

print("Top ORGs:", org_counts.most_common(10))
print("Top LOCs:", loc_counts.most_common(10))


Top ORGs: [('IE', 1795), ('Universidad Autónoma De Madrid', 99), ('Universidad Pontificia Comillas', 88), ('Universidad Complutense de Madrid', 80), ('Universidad Autónoma de Madrid', 79), ('Harvard University', 63), ('Universidad Carlos III de Madrid', 62), ('& Am', 51), ('Universidad Politécnica De Madrid', 41), ('Columbia University', 41)]
Top LOCs: [('Spain', 3490), ('Madrid', 892), ('United States', 464), ('Europe', 202), ('London', 188), ('France', 153), ('Germany', 151), ('Italy', 130), ('Portugal', 114), ('Barcelona', 100)]


In [3]:
output_path = "../data/processed/teachers_db_cleaned.parquet"
df.to_parquet(output_path, index=False)
print(f"Saved cleaned dataset to {output_path}")

NameError: name 'df' is not defined